# Read me

Hello, This notebook is more upgraded than my previous notebook (https://www.kaggle.com/pythonash/parallel-dnn-and-cnn-network-for-beginners).

In this part, you can study how you can concatenate DNN model for handling CSV file and CNN model for handling IMG file.

The data handling procedure will be skipped. If you want to know how it worked in detail, just clik on my previous notebook (https://www.kaggle.com/pythonash/how-to-handle-dataset-for-beginners).

If you have any questions, please leave the comments.

I hope you to gain more imformation about data handling, DNN, CNN, and etc..

## **Knowledge can be improved by being shared.**

Please upvote!!


## [You can learn more skills for handling dataset or neural network.]

### [Parallel combination DNN with CNN] - Pawpularity Contest
 - https://www.kaggle.com/pythonash/parallel-dnn-and-cnn-network-for-beginners
 
### [Image data handling without memory exploded] - Pawpularity Contest
 - https://www.kaggle.com/pythonash/how-to-handle-dataset-for-beginners

### [Data handling & Deep learning] - Titanic competition (best score!!)
 - https://www.kaggle.com/pythonash/how-to-handle-raw-dataset-and-analyze-with-dl
 
### [Deep learning model with SeLU activation function] - Titanic competition
- https://www.kaggle.com/pythonash/selu-activation-function-in-dl

### [Preparing a completed dataset with proper imputation method] - Titanic competition
 - https://www.kaggle.com/pythonash/making-completed-dataset

**Let's start!**

# Just run this code before you set your model.

- This code is for preparing dataset to input at your model.

- The details are described as in my notebook, "Image data handling without memory exploded".

In [ ]:
##################################JUST RUN THIS CODE FOR PREPARING DATASET##################################
import pandas as pd
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

train_csv = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
test_csv = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
submission = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')

os.chdir('../input/petfinder-pawpularity-score/train')

size_data = pd.DataFrame()
for file in os.listdir():
    imgg = cv2.imread(file)
    w,h,c = imgg.shape
    size_data=size_data.append([[w,h,c,imgg.size/3]])
    

train_img = []
for i in os.listdir():
    file = cv2.imread(i)
    file=cv2.resize(file,(64,64), interpolation=cv2.INTER_AREA)
    train_img.append(file/255)


train_img_name = []
for i in os.listdir():
    train_img_name.append(i)


train_csv_data = pd.DataFrame()
for img, name in zip(train_img, train_img_name):
    name=name[:-4]
    location = train_csv[train_csv['Id'] == name].index[0]
    train_csv_data= train_csv_data.append([train_csv.loc[location]])

train_csv_data=train_csv_data.reset_index().drop(['index'],axis=1)

os.chdir('../test')

test_img = []
for i in os.listdir():
    file = cv2.imread(i)
    file=cv2.resize(file,(64,64), interpolation=cv2.INTER_AREA)
    test_img.append(file/255)

test_img_name = []
for i in os.listdir():
    test_img_name.append(i)

test_csv_data = pd.DataFrame()
for img, name in zip(test_img, test_img_name):
    name=name[:-4]
    location = test_csv[test_csv['Id'] == name].index[0]
    test_csv_data= test_csv_data.append([test_csv.loc[location]])
test_csv_data=test_csv_data.reset_index().drop(['index'],axis=1)

train_csv_x = train_csv_data.drop(['Id','Pawpularity'],axis=1)
train_y = train_csv_data['Pawpularity']
test_csv_x = test_csv_data.drop(['Id'],axis=1)

# The comnibation of DNN and CNN

We will use a parallel structure which consists of DNN for CSV and CNN for IMAGE.

The idea is very simple.

1. Set the DNN model and CNN model, respectively.

2. Combinate two models with two outputs.

> In this procedure, you designate the weights for DNN model and CNN model, respectively.

> This model will output two results which are CSV result and IMG result.

> Thus, each result needs its own weight (you can set this weight).


How can it work??

Let's start!

## Set your model

In this part, please read the codes carefully, we will use two input layers.

In [ ]:
##################### CSV FILE INPUT & IMG FILE INPUT LAYER ###########################
csv_input = tf.keras.Input(shape = train_csv_x.shape[1:], name = 'CSV_Input')        ##
img_input = tf.keras.Input(shape = np.array(train_img).shape[1:], name = 'IMG_Input')##
#######################################################################################
                                        ##
                                        ##
                                        ##
##################### CSV FILE HIDDEN LAYER STRUCTURE  ######################################
csv_hidden1 = tf.keras.layers.Dense(8, activation='relu', name='CSV_Hidden1')(csv_input)   ##
csv_hidden2 = tf.keras.layers.Dense(30, activation='relu', name='CSV_Hidden2')(csv_hidden1)##
csv_hidden3 = tf.keras.layers.Dense(50, activation='relu', name='CSV_Hidden3')(csv_hidden2)##
csv_dropout = tf.keras.layers.Dropout(0.5, name ='CSV_Dropout')(csv_hidden3)               ##
csv_hidden4 = tf.keras.layers.Dense(30, activation='relu', name='CSV_Hidden4')(csv_dropout)##
csv_hidden5 = tf.keras.layers.Dense(10, activation='relu', name='CSV_Hidden5')(csv_hidden4)##
#############################################################################################
                                         #
                                         #
                                         #
##################### IMG FILE CONVOLUTIONAL LAYER STRUCTURE  ######################################
img_conv1 = tf.keras.layers.Conv2D(filters = 64, kernel_size = 5, strides= 1, padding = 'same',   ##
                                   activation = 'relu', name='IMG_Conv1')(img_input)              ##
img_pool1 = tf.keras.layers.MaxPool2D(3, name = 'IMG_Pool1')(img_conv1)                           ##
img_conv2 = tf.keras.layers.Conv2D(filters = 128, kernel_size = 4, strides= 1, padding = 'same',  ##
                                   activation = 'relu', name='IMG_Conv2')(img_pool1)              ##
img_conv3 = tf.keras.layers.Conv2D(filters = 128, kernel_size = 4, strides =1, padding = 'same',  ##
                                   activation = 'relu', name='IMG_Conv3')(img_conv2)              ##
img_pool2 = tf.keras.layers.MaxPool2D(3, name = 'IMG_Pool2')(img_conv3)                           ##
img_conv4 = tf.keras.layers.Conv2D(filters = 256, kernel_size = 3, strides =1, padding = 'same',  ##
                                   activation = 'relu', name='IMG_Conv4')(img_pool2)              ##
img_pool3 = tf.keras.layers.MaxPool2D(3, name = 'IMG_Pool3')(img_conv4)                           ##
img_conv5 = tf.keras.layers.Conv2D(filters = 512, kernel_size = 3, strides =1, padding = 'same',  ##
                                   activation = 'relu', name='IMG_Conv5')(img_pool3)              ##
img_conv6 = tf.keras.layers.Conv2D(filters = 512, kernel_size = 3, strides =1, padding = 'same',  ##
                                   activation = 'relu', name='IMG_Conv6')(img_conv5)              ##
img_pool4 = tf.keras.layers.MaxPool2D(2, name = 'IMG_Pool4')(img_conv6)                           ##
img_flatten = tf.keras.layers.Flatten(name = 'IMG_Flatten')(img_pool4)                            ##
img_dense1 = tf.keras.layers.Dense(3000, activation = 'relu', name='IMG_Dense1')(img_flatten)     ##
img_dropout1 = tf.keras.layers.Dropout(0.5, name='IMG_Dropout1')(img_dense1)                      ##
img_dense2 = tf.keras.layers.Dense(3000, activation = 'relu', name='IMG_Dense2')(img_dropout1)    ##
img_dropout2 = tf.keras.layers.Dropout(0.5, name='IMG_Dropout2')(img_dense2)                      ##
####################################################################################################
                                        ##
                                        ##
                                        ##
##################### CSV FILE INPUT & IMG FILE OUTPUT LAYER ############
csv_output = tf.keras.layers.Dense(1, name = 'CSV_Output')(csv_hidden5)##
img_output = tf.keras.layers.Dense(1,name = 'IMG_Output')(img_dropout2)##
#########################################################################
                                        ##
                                        ##
                                        ##
############################################# MODEL SETTING  ####################################################
model = tf.keras.Model(inputs=[csv_input, img_input], outputs=[csv_output, img_output], name='Pythonash_model')##
#################################################################################################################

## model summary

It shows that your model structure, simply.

In [ ]:
model.summary()

## plot your model

This plot shows your model and you can figure out you model structure, intuitively.

You can see that DNN model and CNN model are parallel and two output layer are in this structure.

So, you have to set each weight when you comile your model with the hyper parameter, "loss_weights".

In [ ]:
# move you current directory to back.
os.chdir('../')
os.chdir('../')
os.chdir('../')
tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True, rankdir='TB')

## Compile and fit your model.

In [ ]:
opt = tf.keras.optimizers.Ftrl(learning_rate = 0.003, learning_rate_power = -0.5)
model.compile(loss=['mse','mse'], loss_weights=[0.15, 0.85], optimizer = opt, metrics = ['mape'])

epoch_number = 100

check_1 = tf.keras.callbacks.ModelCheckpoint('pythonash_model.h5', save_best_only=True, verbose=2)

In [ ]:
model.fit( 
    x= [train_csv_x, np.array(train_img)], y = [train_y, train_y], epochs=epoch_number, 
    validation_split=0.3, verbose =2, workers=3, batch_size = 15, validation_batch_size = 15,
    callbacks = [check_1])

## Weighted result

In line 3, you can see the weighted result.

It is as follows in above loss weights.

In [ ]:
best_model = tf.keras.models.load_model('pythonash_model.h5')
csv_result, img_result = best_model.predict([test_csv_x, np.array(test_img)])
final_result = pd.DataFrame(0.15 * csv_result + 0.85 * img_result)
final_result.columns =['Pawpularity']
final_result

## Submit your result.

It has done!

I think it might be easy to you who already know about DNN and CNN.

In [ ]:
for ids, paw in zip(test_csv_data['Id'], final_result['Pawpularity']):
    location = submission[submission['Id'] == ids].index[0]
    submission['Pawpularity'].loc[location] = paw
submission
submission.to_csv('./working/submission.csv',index=False)



# It's your turn!!

You have many opportunities that you can change this model parameters and get your submission score.

I recommend that you change the hyper parameters such as learning_rate, batch_size, activation function, the number of neurons, layers, and so on...

If you get any helps from my notebook, please upvote!!

Fingers crossed!!